In [1]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Download and load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [4]:
# Preprocess the data
X_train = X_train.astype('float32') / 255  # Normalize pixel values between 0 and 1
X_test = X_test.astype('float32') / 255
y_train = to_categorical(y_train, num_classes=10)  # Convert labels to one-hot encoding
y_test = to_categorical(y_test, num_classes=10)

In [5]:
def fuzzy_function(kernel):
    fuzzy_kernel = np.zeros_like(kernel)

    for i in range(kernel.shape[0]):
        for j in range(kernel.shape[1]):
            mu = 0.5  # Mean
            sigma = 0.05  # Standard deviation

            for k in range(kernel.shape[2]):
                for l in range(kernel.shape[3]):
                    weight = kernel[i, j, k, l]
                    fuzzy_weight = np.exp(-(weight - mu) * 2 / (2 * sigma * 2))
                    fuzzy_kernel[i, j, k, l] = fuzzy_weight

    return fuzzy_kernel

In [6]:
# Custom activation layer implementing min(1-a, b)
class MinOneMinusAActivation(keras.layers.Conv2D):
    def __init__(self, filters, kernel_size, **kwargs):
        super(MinOneMinusAActivation, self).__init__(filters, kernel_size, **kwargs)

    def activation(self, x):
        fuzzy_kernel = fuzzy_function(self.kernel)
        return K.minimum(1 - x, fuzzy_kernel)

In [7]:
# Custom activation layer implementing max(1-a, b)
class MaxOneMinusAActivation(keras.layers.Conv2D):
    def __init__(self, filters, kernel_size, **kwargs):
        super(MaxOneMinusAActivation, self).__init__(filters, kernel_size, **kwargs)

    def activation(self, x):
        fuzzy_kernel = fuzzy_function(self.kernel)
        return K.maximum(1 - x, fuzzy_kernel)

In [8]:
# Custom activation function implementing fuzzy thresholding
def fuzzy_threshold(x):
    return K.maximum(K.minimum(x, 1), 0)

In [9]:
# Custom activation function implementing fuzzy softmax
def fuzzy_softmax(x):
    max_val = K.max(x, axis=-1, keepdims=True)
    e = K.exp(x - max_val)
    s = K.sum(e, axis=-1, keepdims=True)
    return e / s

In [10]:
# Create a sequential model
model = keras.models.Sequential()
# Add Conv2D layers with custom activations
model.add(MinOneMinusAActivation(32, kernel_size=(3, 3), activation=fuzzy_threshold, input_shape=(32, 32, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(MaxOneMinusAActivation(32, kernel_size=(3, 3), activation=fuzzy_threshold))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))  # Add Dropout layer

model.add(keras.layers.Flatten())

# Add fully connected layers for classification
model.add(keras.layers.Dense(256, activation=fuzzy_threshold))
model.add(keras.layers.Dense(128, activation=fuzzy_threshold))
model.add(keras.layers.Dense(64, activation=fuzzy_threshold))
model.add(keras.layers.Dense(10, activation=fuzzy_softmax))

In [17]:
 # Compile the model using categorical cross-entropy as loss (not fuzzy loss for simplicity)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=256),
                    epochs=30,
                    validation_data=(X_test, y_test),
                    verbose=1,
                    )


Epoch 1/30
196/196 [==============================] - 96s 472ms/step - loss: 1.7852 - accuracy: 0.3460 - val_loss: 1.5041 - val_accuracy: 0.4634
Epoch 2/30
196/196 [==============================] - 87s 443ms/step - loss: 1.5282 - accuracy: 0.4445 - val_loss: 1.3947 - val_accuracy: 0.5048
Epoch 3/30
196/196 [==============================] - 88s 450ms/step - loss: 1.4299 - accuracy: 0.4822 - val_loss: 1.2951 - val_accuracy: 0.5387
Epoch 4/30
196/196 [==============================] - 87s 438ms/step - loss: 1.3596 - accuracy: 0.5089 - val_loss: 1.2368 - val_accuracy: 0.5580
Epoch 5/30
196/196 [==============================] - 87s 441ms/step - loss: 1.3048 - accuracy: 0.5323 - val_loss: 1.1486 - val_accuracy: 0.5963
Epoch 6/30
196/196 [==============================] - 84s 430ms/step - loss: 1.2640 - accuracy: 0.5510 - val_loss: 1.1048 - val_accuracy: 0.6088
Epoch 7/30
196/196 [==============================] - 84s 428ms/step - loss: 1.2254 - accuracy: 0.5610 - val_loss: 1.0437 - val_ac

In [18]:
# Évaluez le modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

# Affichez les résultats
print(f'Loss on test set: {loss:.4f}')
print(f'Accuracy on test set: {accuracy * 100:.2f}%')


313/313 [==============================] - 6s 19ms/step - loss: 0.8205 - accuracy: 0.7129
Loss on test set: 0.8205
Accuracy on test set: 71.29%
